# Portfolio Journal Generator

This notebook demonstrates how to generate a concise portfolio journal entry using LLMs based on trading data from Robinhood (via SnapTrade) and Discord message sentiment. It uses the `journal_generator.py` module for production use.

In [ ]:
import pandas as pd
import json
import os
import sys
import re
from pathlib import Path
from datetime import datetime

# Add the src directory to the path so we can import from src
sys.path.append('..')

# Import our journal generator module
from src.journal_generator import (
    load_positions,
    load_discord_messages,
    load_prices,
    generate_portfolio_journal
)

# Define paths with Path objects
BASE_DIR = Path().resolve().parent
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

# File paths
DISCORD_CSV = RAW_DIR / "discord_msgs.csv"
POSITIONS_CSV = RAW_DIR / "positions.csv"
PRICES_CSV = RAW_DIR / "prices.csv"

# Load data
positions_df = load_positions(POSITIONS_CSV)
messages_df = load_discord_messages(DISCORD_CSV)
prices_df = load_prices(PRICES_CSV)

## Add New Discord Messages

This cell allows you to quickly add a Discord message to your dataset.

In [ ]:
# Import the function to add Discord messages to the CSV
from src.data_collector import append_discord_message_to_csv
from src.journal_generator import extract_ticker_and_text_pairs

# Function to add a message and reload the dataset
def add_message_and_reload(message_text):
    # Call the improved function to safely append to CSV
    append_discord_message_to_csv(message_text)
    
    # Reload messages dataframe
    global messages_df
    messages_df = load_discord_messages(DISCORD_CSV)
    print(f"✅ Reloaded Discord messages - now have {len(messages_df)} records")

# Example usage - Uncomment and modify to add a message
'''
message = """$ASML
decade-long lead over competitors, supported by an unrivaled technical advantage and proprietary supply chain partnerships with companies like Zeiss (precision optics) and Trumpf (laser systems).
Gross margin: 51%; net margin: 28%. Balance sheet: More cash than debt.
20% of revenue comes from China. Temporary spikes (up to 50%) occurred recently due to preemptive purchases amid export restrictions. Taiwan-China tensions pose risks, but if conflict occurs, ASML's role in rebuilding chip infrastructure globally would likely mitigate long-term damage.
ASML collaborates with 4,000+ suppliers, many exclusively serving ASML, a key competitive moat.
ASML's next-generation High NA EUV machines (costing €400M/unit) will drive future growth
ASML expects significant revenue growth in 2025 after a transition year in 2024
Long-term guidance (2030 targets):
Revenue: €44B (bear), €52B (mid), €60B (bull).
Gross margin improvement: Potential increase from 51% to 60%.
P/E ratios (20-30x) are below ASML's historical average

Challenges:
Cyclical semiconductor demand affects short-term revenue growth (15% projected for 2024 due to weakened demand from Intel and Samsung)."""

add_message_and_reload(message)
'''

## Add Multiple Stock Analysis Messages

This cell helps you add multiple stock analyses at once (like the thread you shared).

In [ ]:
def process_stock_analysis_thread(text):
    """Split a multi-stock analysis thread into individual messages and add each one"""
    # Use the improved ticker extraction function that handles positions properly
    ticker_text_pairs = extract_ticker_and_text_pairs(text)
    
    if not ticker_text_pairs:
        print("No stock sections detected. Make sure ticker symbols are in $TICKER format.")
        return
    
    # Process each ticker-text pair
    for ticker, section_text in ticker_text_pairs:
        add_message_and_reload(section_text)
    
    print(f"✅ Added {len(ticker_text_pairs)} stock analysis messages")

# Example - Uncomment to use
'''
thread_text = """$ASML
decade-long lead over competitors, supported by an unrivaled technical advantage and proprietary supply chain partnerships with companies like Zeiss (precision optics) and Trumpf (laser systems).
Gross margin: 51%; net margin: 28%. Balance sheet: More cash than debt.
20% of revenue comes from China. Temporary spikes (up to 50%) occurred recently due to preemptive purchases amid export restrictions. Taiwan-China tensions pose risks, but if conflict occurs, ASML's role in rebuilding chip infrastructure globally would likely mitigate long-term damage.
ASML collaborates with 4,000+ suppliers, many exclusively serving ASML, a key competitive moat.
ASML's next-generation High NA EUV machines (costing €400M/unit) will drive future growth
ASML expects significant revenue growth in 2025 after a transition year in 2024
Long-term guidance (2030 targets):
Revenue: €44B (bear), €52B (mid), €60B (bull).
Gross margin improvement: Potential increase from 51% to 60%.
P/E ratios (20-30x) are below ASML's historical average

Challenges:
Cyclical semiconductor demand affects short-term revenue growth (15% projected for 2024 due to weakened demand from Intel and Samsung).
$HIMS
Mental health only getting more $$ and demand, Viagra 📈 , men's hair loss :up1: , subscription model that's relatively hard to cancel/talk out of...  Lots of comp to come, so probably taper at highs
2M+ subscribers growing 40%+ annually 75%+ gross margins, 15%+ free cash flow, no debt, and consistent revenue growth (65% this year, projected 20%+ annually). Trading at ~20x 2025 FCF,well-positioned for long-term success under its founder-CEO, the largest shareholder."""

process_stock_analysis_thread(thread_text)
'''

## Generate Journal Entry

Now we'll generate our journal entry using the `generate_portfolio_journal` function.

In [ ]:
# Simply call the function from our module to generate the journal
journal_entry = generate_portfolio_journal(
    positions_path=POSITIONS_CSV,
    discord_path=DISCORD_CSV,
    prices_path=PRICES_CSV,
    output_dir=PROCESSED_DIR
)

## Other Ways to Run

You can also run the journal generator as a script:

In [ ]:
# This is equivalent to running the script directly
!python -m src.journal_generator